In [14]:
#Cours rapide sur les class

class MyClass():                        # correspond à une "boite à outils" qui contient des méthodes

    def __init__(self, nb1, nb2):       # constructeur necessaire pour initialiser les attributs (self correspon à l'objet construit)
        self.nb1 = nb1
        self.nb2 = nb2

    def substract(self,):                # création d'un méthode (besoin de self pour pouvoir accéder aux attributs)

        return self.nb1 - self.nb2
        

In [15]:
objet1=MyClass(2,3)

objet1.substract()

-1

In [ ]:
%matplotlib inline


[Learn the Basics](intro.html) ||
**Quickstart** ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
[Build Model](buildmodel_tutorial.html) ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Quickstart
This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.

## Working with data
PyTorch has two [primitives to work with data](https://pytorch.org/docs/stable/data.html):
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html),
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like
CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

100%|██████████| 26421880/26421880 [00:03<00:00, 7567204.83it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 495975.07it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 7080051.83it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5943373.79it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element
in the dataloader iterable will return a batch of 64 features and labels.



In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")    #correspond aux dimensions du Tensor : 64 images, 1 channel, 28x28 pixels
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Read more about [loading data in PyTorch](data_tutorial.html).




--------------




## Creating Models
To define a neural network in PyTorch, we create a class that inherits
from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). We define the layers of the network
in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. To accelerate
operations in the neural network, we move it to the GPU if available.



In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):                     #création d'une classe qui hérite de nn.Module
    def __init__(self):                             #Definition des briques de la NN (tous les poids, couches, fonctions d'activation)
        super().__init__()                          #appel du constructeur de la classe mère (nn.Module)
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(     #Enchainement d'opérations matricielles et non linéaires
            nn.Linear(28*28, 512),                  #multiplication matricielle de 28*28 par 512
            #X [N, 1*28*28] -> [N, 512]
            nn.ReLU(),                              #fonction d'activation de derivée 0 ou 1
            #X [N, 512] -> [N, 512] avec 0 si négatf et x si positf
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
            #X [N, 512] -> [N, 10] pour avoir comme sorti des probabilités ( mais pas encore forcment normalisées !)
        )                                           #Enchinement de linéarité et de non linéarité pour vrt avoir plus plusisuers couches

    def forward(self, x):               #Dans quel sens on imbrique les briques ; c'est cette fonction qui est appelée lors de l'appel de la NN
        #X [N, C, H, W] -> [N, 1, 28, 28]]                          
        x = self.flatten(x)
        #X [N, C*H*W] -> [N, 1*28*28]      l'axe 0 correspond aux images, l'axe 1 aux pixels (convention PyTorch))                   
        logits = self.linear_relu_stack(x)          #modifie les valeurs de x pour normaliser les probabilités avec la fonction softmax
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Read more about [building neural networks in PyTorch](buildmodel_tutorial.html).




--------------




## Optimizing the Model Parameters
To train a model, we need a [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions)
and an [optimizer](https://pytorch.org/docs/stable/optim.html).



In [5]:
loss_fn = nn.CrossEntropyLoss()     #fonction de perte pour calculer l'erreur entre la sortie de la NN et la sortie attendue, ici la fonction de perte CrossEntropyLoss
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)    #optimiseur pour mettre à jour les poids de la NN

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and
backpropagates the prediction error to adjust the model's parameters.



In [6]:
def train(dataloader, model, loss_fn, optimizer):        # regler les poids de la NN pour qu'elle soit capable de prédire la bonne classe
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):            
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)                                 #appel de la NN pour recuperer les prédictions
        loss = loss_fn(pred, y)                         #calcul de l'erreur entre les prédictions et les valeurs attendues

        # Backpropagation
        optimizer.zero_grad()                           #remise à zéro des gradients
        loss.backward()                                 #calcul des gradients avec la fonction de perte calculée précédemment (calcul de la dérivée de la fonction de perte pour chaque poids, pour savoir si il faut augmenter ou diminuer le poids)
        optimizer.step()                                #mise à jour des poids

        if batch % 100 == 0:                                            #affichage de l'erreur toutes les 100 images
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.



In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()                                #mode evaluation pour la NN (pas de mise à jour des poids)
    test_loss, correct = 0, 0
    with torch.no_grad():                        #pas de calcul des gradients pour la phase de test pour gain de temps et de mémoire
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)     
            test_loss += loss_fn(pred, y).item()        
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()     #comptage du nombre de prédictions correctes
    test_loss /= num_batches                #calcul de l'erreur moyenne
    correct /= size                         #calcul du taux de réussite
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [8]:
epochs = 5                          #nombre d'itérations pour l'apprentissage
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304452  [    0/60000]
loss: 2.294509  [ 6400/60000]
loss: 2.276471  [12800/60000]
loss: 2.271077  [19200/60000]
loss: 2.258825  [25600/60000]
loss: 2.231685  [32000/60000]
loss: 2.236426  [38400/60000]
loss: 2.207886  [44800/60000]
loss: 2.202768  [51200/60000]
loss: 2.176368  [57600/60000]
Test Error: 
 Accuracy: 50.6%, Avg loss: 2.172319 

Epoch 2
-------------------------------
loss: 2.176753  [    0/60000]
loss: 2.171722  [ 6400/60000]
loss: 2.121804  [12800/60000]
loss: 2.138791  [19200/60000]
loss: 2.101194  [25600/60000]
loss: 2.040613  [32000/60000]
loss: 2.068672  [38400/60000]
loss: 1.997792  [44800/60000]
loss: 2.001111  [51200/60000]
loss: 1.937239  [57600/60000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 1.936011 

Epoch 3
-------------------------------
loss: 1.958468  [    0/60000]
loss: 1.939251  [ 6400/60000]
loss: 1.834183  [12800/60000]
loss: 1.873890  [19200/60000]
loss: 1.782360  [25600/60000]
loss: 1.719778  [32000/600

Read more about [Training your model](optimization_tutorial.html).




--------------




## Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

## Loading Models

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it.



In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

This model can now be used to make predictions.



In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Read more about [Saving & Loading your model](saveloadrun_tutorial.html).


